### Run tests for warped p-variation

In [1]:
# !python tests.py

In [1]:
import sys
sys.path.insert(0, '../backend')
sys.path.insert(0, '../../data')

### Import libraries

In [2]:
import numpy as np
import pandas as pd
import time
import math
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from generalized_dtw import Warp
from pvar_backend import *
from brute_force_warped_pvar import *
from pvar_tools import *
from branch_and_bound_warped_pvar import *

from transformers import *

In [3]:
# import sys, importlib
# importlib.reload(sys.modules['brute_force_warped_pvar'])

# Example with simple signature p-variation

In [3]:
idx = np.linspace(0, 6.28, 10)
path = np.sin(idx)
path = AddTime().fit_transform([path])[0]

In [4]:
p_variation_path(path=path, p=1.5, depth=2, norm='l1', pth_root=False)

(15.747476636533667, [0, 2, 4, 7, 9])

In [5]:
p_variation_path(path=path, p=1.5, depth=2, norm='l1', pth_root=True)

(6.282617916215064, [0, 2, 4, 7, 9])

# Example with signature p-variation distance

### We assume that $path_1$ and $path_2$ have already been aligned according to a warp

In [6]:
idx = np.linspace(0, 6.28, 10)
path1 = np.sin(idx)
path2 = np.cos(idx)
path1 = AddTime().fit_transform([path1])[0]
path2 = AddTime().fit_transform([path2])[0]

In [7]:
d = p_variation_distance(path1, path2, p=1.5, depth=2, norm='l1', optim_partition=True, pvar_advanced=False, pth_root=True)
print('distance with dynamic programming: {}'.format(d))

distance with dynamic programming: (8.330463771808713, [0, 4, 9])


In [8]:
d = p_variation_distance(path1, path2, p=1.5, depth=2, norm='l1', optim_partition=True, pvar_advanced=False, pth_root=False)
print('distance with dynamic programming: {}'.format(d))

distance with dynamic programming: (24.043836720083362, [0, 4, 9])


# Example with warped p-var package

In [9]:
idx = np.linspace(0, 6.28, 6)
path1 = np.sin(idx)
path2 = np.cos(idx)
path1 = AddTime().fit_transform([path1])[0]
path2 = AddTime().fit_transform([path2])[0]

### Compute the warped signature p-variation distance between $path_1$ and $path_2$

### 1) Sequential calculations

In [12]:
LP_seq = BruteForceWarpedPvar(path1, path2, p=1.5, depth=2, norm='l1', augment=False, add_time=False, parallelise=False, 
                              optim_partition=True, pvar_advanced=False, pth_root=False)

number of warping paths to explore: 252
time to find all possible paths: 0.13


Loop over all warps: 100%|████████████████████████████████████████████████████████| 1682/1682 [00:02<00:00, 752.00it/s]


total time for brute force with DP sequentially: 2.24 s


In [13]:
LP_seq_pth_root = BruteForceWarpedPvar(path1, path2, p=1.5, depth=2, norm='l1', augment=False, add_time=False, 
                                       parallelise=False, optim_partition=True, pvar_advanced=False, pth_root=True)

number of warping paths to explore: 252
time to find all possible paths: 0.13


Loop over all warps: 100%|████████████████████████████████████████████████████████| 1682/1682 [00:02<00:00, 795.52it/s]


total time for brute force with DP sequentially: 2.12 s


### global answers

In [20]:
print('warping distance with dynamic programming: {} along {}. Best partiton:{}'.format(LP_seq.warped_pvar, 
                                                                                        LP_seq.best_warp, 
                                                                                        LP_seq.best_partition))

warping distance with dynamic programming: 14.76997293868892 along [(0, 0), (1, 0), (2, 0), (3, 1), (4, 2), (4, 3), (4, 4), (5, 5), (5, 5), (5, 5), (5, 5)]. Best partiton:[0, 1, 3, 4, 10]


In [21]:
print('warping distance with dynamic programming: {} along {}. Best partition: {}'.format(LP_seq_pth_root.warped_pvar, 
                                                                                      LP_seq_pth_root.best_warp, 
                                                                                      LP_seq_pth_root.best_partition))

warping distance with dynamic programming: 6.019861041460879 along [(0, 0), (1, 0), (2, 0), (3, 1), (4, 2), (4, 3), (4, 4), (5, 5), (5, 5), (5, 5), (5, 5)]. Best partition: [0, 1, 3, 4, 10]


### 2) Computations in parallel

In [22]:
LP = BruteForceWarpedPvar(path1, path2, p=1.5, depth=2, norm='l1', augment=False, add_time=False, parallelise=True, 
                          optim_partition=True, pvar_advanced=False, pth_root=False)

number of warping paths to explore: 252
time to find all possible paths: 0.15
Using all available cores, i.e. 4
total time for brute force with DP in Parallel: 3.28 s
